# Pipeline d'entraînement des modèles NER avec spaCy (Google colab / GPU)

Referencias:
*   NER4ARCHIVES : https://github.com/NER4Archives-project/spacy_ner_training_pipeline
*   Spacy transformers : https://www.youtube.com/watch?v=Y_N_AO39rRg
*   Spacy tranformers : https://towardsdatascience.com/how-to-fine-tune-bert-transformer-with-spacy-3-6a90bfe57647
*   Spancat Spacy : https://explosion.ai/blog/spancat


##Instalaciones

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#!pip uninstall -y spacy setuptools wheel spacy-huggingface-hub spacy-transformers cupy cupy-cuda113 cupy-cuda11x cupy-cuda12x

In [3]:
!pip install -U pip setuptools wheel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.1.0
    Uninstalling setuptools-75.1.0:
      Successfully uninstalled setuptools-75.1.0
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [4]:
!pip install -U 'spacy[cuda12x,transformers,lookups]'
#attention, ne fonctionne pas avec d'autres version de cuda, seul 12x pour compatibilité (le 28/11/2024)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 24.7 MB/s eta 0:00:00
  Attempting uninstall: blis
    Found existing installation: blis 0.7.11
    Uninstalling blis-0.7.11:
      Successfully uninstalled blis-0.7.11
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1
  Attempting uninstall: thinc
    F

In [1]:
!python -m spacy download fr_dep_news_trf
!python -m spacy download fr_core_news_sm
#!python -m spacy download es_dep_news_trf
#!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.7/397.7 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 34.1 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<6.0.0dev,>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
✔ Download and installation successful
You can now load the package via spacy.load('fr_dep_news_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies.

##Verificaciones y definiciones

Vérifier Spacy en version 3.x et l'utilisation du GPU

In [15]:
import spacy
import torch
torch.cuda.empty_cache()

print('GPU State : ', spacy.prefer_gpu()) # doit renvoyer True
try:
  print('GPU required : ', spacy.require_gpu(0)) # doit renvoyer True
except ValueError:
  print("No GPU devices detected")
print('Cuda available : ', torch.cuda.is_available()) # doit renvoyer True
print('SPACY VERSION : ', spacy.__version__)

GPU State :  True
GPU required :  True
Cuda available :  True
SPACY VERSION :  3.8.4


GPU et RAM Premium

In [16]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jan 16 13:56:59 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0              31W /  70W |    103MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [17]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [18]:
!python -m spacy info

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)

============================== Info about spaCy ==============================

spaCy version    3.8.4                         
Location         /usr/local/lib/python3.11/dist-packages/spacy
Platform         Linux-6.1.85+-x86_64-with-glibc2.35
Python version   3.11.11                       
Pipelines        

In [19]:
def remplacer_mots(fichier_source, mots_a_remplacer, nouveaux_mots, fichier_destination):
    try:
        # Vérifier que les deux listes ont la même longueur
        if len(mots_a_remplacer) != len(nouveaux_mots):
            raise ValueError("Les listes 'mots_a_remplacer' et 'nouveaux_mots' doivent avoir la même longueur.")

        # Ouvrir le fichier source en lecture
        with open(fichier_source, 'r', encoding='utf-8') as file:
            contenu = file.read()

        # Remplacer chaque mot dans la liste
        for mot_a_remplacer, nouveau_mot in zip(mots_a_remplacer, nouveaux_mots):
            contenu = contenu.replace(mot_a_remplacer, nouveau_mot)

        # Écrire le nouveau contenu dans le fichier de destination
        with open(fichier_destination, 'w', encoding='utf-8') as file:
            file.write(contenu)

        print(f"Les mots {mots_a_remplacer} ont été remplacés par {nouveaux_mots} dans '{fichier_destination}'.")

    except FileNotFoundError:
        print(f"Le fichier '{fichier_source}' n'a pas été trouvé.")
    except ValueError as ve:
        print(ve)
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")

# Exemple d'utilisation
#mots_a_remplacer = ['ancien_mot1', 'ancien_mot2']
#nouveaux_mots = ['nouveau_mot1', 'nouveau_mot2']
#remplacer_mots('exemple.txt', mots_a_remplacer, nouveaux_mots, 'nouveau_fichier.txt')


##Preparacion de los datos de entrenamiento del modelo NER

### 10e parte sobre (/30) para cada texto excepto 1, 7 y 15

In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/10e_part_de_30_annot"

es_annot_data_topeditor_12_part_10_ok.conll  fr_annot_data_topeditor_18_part_10_ok.conll
es_annot_data_topeditor_17_part_10_ok.conll  fr_annot_data_topeditor_19_part_10_ok.conll
es_annot_data_topeditor_18_part_10_ok.conll  fr_annot_data_topeditor_20_part_10_ok.conll
es_annot_data_topeditor_19_part_10_ok.conll  fr_annot_data_topeditor_2_part_10_ok.conll
fr_annot_data_topeditor_10_part_10_ok.conll  fr_annot_data_topeditor_3_part_10_ok.conll
fr_annot_data_topeditor_11_part_10_ok.conll  fr_annot_data_topeditor_4_part_10_ok.conll
fr_annot_data_topeditor_12_part_10_ok.conll  fr_annot_data_topeditor_5_part_10_ok.conll
fr_annot_data_topeditor_13_part_10_ok.conll  fr_annot_data_topeditor_6_part_10_ok.conll
fr_annot_data_topeditor_14_part_10_ok.conll  fr_annot_data_topeditor_8_part_10_ok.conll
fr_annot_data_topeditor_16_part_10_ok.conll  fr_annot_data_topeditor_9_part_10_ok.conll
fr_annot_data_topeditor_17_part_10_ok.conll


In [ ]:
!cat /content/drive/MyDrive/"Colab Notebooks"/TopEditor/data/conll/10e_part_de_30_annot/* > \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_10e_sur_30_text_2-6_8-14_16-20_modif_OK.conll"

In [ ]:
!wc -l "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_10e_sur_30_text_2-6_8-14_16-20_modif_OK.conll"

7456 /content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_10e_sur_30_text_2-6_8-14_16-20_modif_OK.conll


### 1e partes sobre (/30) para cada texto excepto 1, 7 y 15

In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/1e_part_de_30_annot"

es_annot_data_topeditor_11_part_1_modif_OK.conll
es_annot_data_topeditor_18_part_1_modif_OK.conll
es_annot_data_topeditor_2_part_1_modif_OK.conll
es_annot_data_topeditor_3_part_1_modif_OK.conll
es_annot_data_topeditor_4_part_1_modif_OK.conll
es_annot_data_topeditor_9_part_1_modif_OK.conll
es_fr_annot_data_topeditor_10_part_1_modif_OK.conll
fr_annot_data_topeditor_12_part_1_modif_OK.conll
fr_annot_data_topeditor_13_part_1_modif_OK.conll
fr_annot_data_topeditor_14_part_1_modif_OK.conll
fr_annot_data_topeditor_16_part_1_modif_OK.conll
fr_annot_data_topeditor_19_part_1_modif_OK.conll
fr_annot_data_topeditor_20_part_1_modif_OK.conll
fr_annot_data_topeditor_5_part_1_modif_OK.conll
fr_annot_data_topeditor_6_part_1_modif_OK.conll
fr_annot_data_topeditor_8_part_1_modif_OK.conll
fr_es_annot_data_topeditor_17_part_1_modif_OK.conll


In [ ]:
!cat /content/drive/MyDrive/"Colab Notebooks"/TopEditor/data/conll/1e_part_de_30_annot/* > \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1e_sur_30_text_2-6_8-14_16-20_modif_OK.conll"


In [ ]:
!wc -l "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1e_sur_30_text_2-6_8-14_16-20_modif_OK.conll"

7492 /content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1e_sur_30_text_2-6_8-14_16-20_modif_OK.conll


### partes diversas sobre tres (/3) para los textos 1 y 7

In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/textes_7_1/"

annot_data_topeditor_1_1etier_modif_OK.conll  annot_data_topeditor_7_2etier_modif_OK.conll
annot_data_topeditor_1_2etier_modif_OK.conll  annot_data_topeditor_7_3etier_modif_OK.conll
annot_data_topeditor_1_3etier_modif_OK.conll


In [ ]:
!cat \
/content/drive/MyDrive/"Colab Notebooks"/TopEditor/data/conll/textes_7_1/* > \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1_all_7_3tier_modif_OK.conll"


In [ ]:
!wc -l "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1_all_7_3tier_modif_OK.conll"

17254 /content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1_all_7_3tier_modif_OK.conll


###Limpieza de las anotaciones disponibles para el entrenamiento

Eliminando errores en las categories et aquellas con pocas ocurrencias

In [ ]:
remplacer_mots( \
  '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1_all_7_3tier_modif_OK.conll', \
  ['B-Adefinir30', 'I-Adefinir30','B-expresion_medidas','I-expresion_medidas','B-Obras','I-Obras', 'B-Religion', 'I-Religion', 'B-estado_casa', 'I-estado_casa', 'B-elemento_arquitectura', 'I-elemento_arquitectura'], \
  ['O', 'O', 'B-medidas', 'I-medidas', 'O','O', 'O', 'O', 'O', 'O', 'O', 'O'], \
  '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1_all_7_3tier_modif_OK_cleancat.conll')

remplacer_mots( \
  '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1e_sur_30_text_2-6_8-14_16-20_modif_OK.conll', \
  ['B-Adefinir30', 'I-Adefinir30','B-expresion_medidas','I-expresion_medidas','B-Obras','I-Obras', 'B-Religion', 'I-Religion', 'B-estado_casa', 'I-estado_casa', 'B-elemento_arquitectura', 'I-elemento_arquitectura'], \
  ['O', 'O', 'B-medidas', 'I-medidas', 'O','O', 'O','O', 'O', 'O', 'O', 'O'], \
  '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1e_sur_30_text_2-6_8-14_16-20_modif_OK_cleancat.conll')

remplacer_mots( \
  '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_10e_sur_30_text_2-6_8-14_16-20_modif_OK.conll', \
  ['B-Adefinir30', 'I-Adefinir30','B-expresion_medidas','I-expresion_medidas','B-Obras','I-Obras', 'B-Religion', 'I-Religion', 'B-estado_casa', 'I-estado_casa', 'B-elemento_arquitectura', 'I-elemento_arquitectura'], \
  ['O', 'O', 'B-medidas', 'I-medidas', 'O','O', 'O','O', 'O', 'O', 'O', 'O'], \
  '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_10e_sur_30_text_2-6_8-14_16-20_modif_OK_cleancat.conll')



Les mots ['B-Adefinir30', 'I-Adefinir30', 'B-expresion_medidas', 'I-expresion_medidas', 'B-Obras', 'I-Obras', 'B-Religion', 'I-Religion', 'B-estado_casa', 'I-estado_casa', 'B-elemento_arquitectura', 'I-elemento_arquitectura'] ont été remplacés par ['O', 'O', 'B-medidas', 'I-medidas', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] dans '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1_all_7_3tier_modif_OK_cleancat.conll'.
Les mots ['B-Adefinir30', 'I-Adefinir30', 'B-expresion_medidas', 'I-expresion_medidas', 'B-Obras', 'I-Obras', 'B-Religion', 'I-Religion', 'B-estado_casa', 'I-estado_casa', 'B-elemento_arquitectura', 'I-elemento_arquitectura'] ont été remplacés par ['O', 'O', 'B-medidas', 'I-medidas', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] dans '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1e_sur_30_text_2-6_8-14_16-20_modif_OK_cleancat.conll'.
Les mots ['B-Adefinir30', 'I-Adefinir30', 'B-expresion_medidas', 'I-expresion_me

###Agregando todas las anotaciones disponibles para el entrenamiento

In [ ]:
!cat \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_10e_sur_30_text_2-6_8-14_16-20_modif_OK_cleancat.conll" \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1e_sur_30_text_2-6_8-14_16-20_modif_OK_cleancat.conll" \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_1_all_7_3tier_modif_OK_cleancat.conll" > \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/train_modif_OK_cleancat.conll"


In [ ]:
!python -m spacy convert \
 "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/train_modif_OK_cleancat.conll" \
 "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/bin_containers/" \
 -t spacy -c iob -b fr_core_news_sm #default, hay mas texto en fr que en ES


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Generated output file (530 documents): /content/drive/MyDrive/Colab
Notebooks/TopEditor/data/bin_containers/train_modif_OK_cleancat.spacy


##Preparacion de los datos de validacion del modelo NER


In [ ]:
remplacer_mots( \
  '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/validation/annot_data_topeditor_7_1etier_modif_OK.conll', \
  ['B-Adefinir30', 'I-Adefinir30','B-expresion_medidas','I-expresion_medidas', 'B-Obras', 'I-Obras', 'B-Religion', 'I-Religion',  'B-estado_casa', 'I-estado_casa', 'B-elemento_arquitectura', 'I-elemento_arquitectura'], \
  ['O', 'O', 'B-medidas', 'I-medidas', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], \
  '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_7_1etier_modif_OK_cleancat.conll')

Les mots ['B-Adefinir30', 'I-Adefinir30', 'B-expresion_medidas', 'I-expresion_medidas', 'B-Obras', 'I-Obras', 'B-Religion', 'I-Religion', 'B-estado_casa', 'I-estado_casa', 'B-elemento_arquitectura', 'I-elemento_arquitectura'] ont été remplacés par ['O', 'O', 'B-medidas', 'I-medidas', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] dans '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_7_1etier_modif_OK_cleancat.conll'.


In [ ]:
!python -m spacy convert \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/annot_data_topeditor_7_1etier_modif_OK_cleancat.conll" \
 "/content/drive/MyDrive/Colab Notebooks/TopEditor/data/bin_containers/" \
 -t spacy -c iob -b fr_core_news_sm


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
⚠ No sentence boundaries found. Use `-s` to automatically segment
sentences.
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Generated output file (62 documents): /content/drive/MyDrive/Colab
Notebooks/TopEditor/data/bin_containers/annot_data_topeditor_7_1etier_modif_OK_cleancat.spacy


## Copiar los datos de entrenamiento y validacion en local


train_modif_OK_cleancat.conll

FR_train_modif_OK_cleancat.conll

ES_train_modif_OK_cleancat.conll

In [20]:
!cp \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/bin_containers/train_modif_OK_cleancat.spacy" \
./train.spacy

!cp \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/bin_containers/annot_data_topeditor_7_1etier_modif_OK_cleancat.spacy" \
./dev.spacy

##Preparacion del entrenamiento del modelo NER

Inicializacion de la configuration del entrenamiento

**ES : preferencia un transformer ES**

In [ ]:
!python -m spacy init fill-config \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/config/config_gpu_trf_es_config.cfg" \
./config.cfg


/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


**FR : preferencia un transformer FR**

In [21]:
!python -m spacy init fill-config \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/config/base_config_fr_trf_ner.cfg" \
./config.cfg


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Debugging

In [22]:
!python -m spacy debug config config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(

============================= Config validation =============================
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(

===================== Config validation for [initialize] =====================

====================== Config validation for [training] ======================
✔ Config is valid


In [23]:
!python -m spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(

============================ Data file validation ============================
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` i

##Entrenamiento del modelo NER

In [24]:
!python -m spacy train --gpu-id 0 config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy \
--output "/content/drive/MyDrive/Colab Notebooks/TopEditor/NERmodels/model_spacy_gpu_trf_fr_text_all_ALLcat/"

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
ℹ Saving to output directory: /content/drive/MyDrive/Colab
Notebooks/TopEditor/NERmodels/model_spacy_gpu_trf_fr_text_all_ALLcat
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings

Salida entrenamiento modelo 08/12/2024 (textos: 1, 7, 1e partes, 10e partes)

```
============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE
---  ------  -------------  --------  ------  ------  ------  ------
/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
  0       0        1229.86    623.29    0.00    0.00    0.00    0.00
  9     200      243992.98  188920.02    9.08   11.49    7.50    0.09
 18     400      269496.65  92049.60   36.24   48.09   29.07    0.36
 28     600      220346.08  66540.57   62.32   68.56   57.13    0.62
 37     800       94611.03  31940.58   77.12   77.78   76.48    0.77
 47    1000       38140.16  12890.20   79.25   79.81   78.70    0.79
 57    1200       16153.99   5171.19   77.95   79.37   76.57    0.78
 67    1400       10873.91   2965.42   79.14   79.59   78.70    0.79
 76    1600       11108.96   2223.51   80.34   81.64   79.07    0.80
 85    1800        5372.71   1203.18   80.71   81.24   80.19    0.81
 95    2000       10340.43   1064.32   79.57   80.36   78.80    0.80
105    2200        2487.83    541.38   79.96   80.68   79.26    0.80
114    2400        2925.59    421.75   79.96   80.11   79.81    0.80
124    2600        2519.99    390.31   82.07   83.24   80.93    0.82
133    2800        1280.99    219.75   81.06   81.48   80.65    0.81
143    3000        1233.54    191.08   82.96   84.39   81.57    0.83
152    3200         884.18    157.61   81.84   83.17   80.56    0.82
162    3400         730.40    130.18   81.35   82.35   80.37    0.81
172    3600         894.55    172.61   80.83   82.47   79.26    0.81
181    3800         996.62    128.72   81.28   82.60   80.00    0.81
191    4000         540.05     92.30   80.79   81.98   79.63    0.81
200    4200         428.71     68.52   80.93   81.50   80.37    0.81
210    4400         787.17     70.41   80.58   81.46   79.72    0.81
219    4600         783.97    114.53   81.12   81.89   80.37    0.81
✔ Saved pipeline to output directory
/content/drive/MyDrive/Colab
Notebooks/TopEditor/NERmodels/model_spacy_gpu_trf_fr_text_all_ALLcat/model-last

```



Salida (ultimo training - courrier commun)
```
============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE
---  ------  -------------  --------  ------  ------  ------  ------
/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
  0       0        2021.50    386.76    0.00    0.00    0.00    0.00
 18     200      508678.21  118390.61   67.19   67.28   67.10    0.67
 37     400       30238.27  16284.90   75.05   75.25   74.84    0.75
 56     600        4117.54   2095.89   75.18   75.25   75.12    0.75
 74     800        1480.86   1111.03   75.67   77.00   74.38    0.76
 93    1000         349.52    367.49   76.15   76.83   75.48    0.76
112    1200         351.98    276.34   75.40   74.95   75.85    0.75
130    1400         514.87    233.57   76.89   79.26   74.65    0.77
149    1600         194.93    154.15   76.55   78.43   74.75    0.77
168    1800         125.96    130.35   76.67   77.80   75.58    0.77
186    2000          52.67     68.54   77.36   79.04   75.76    0.77
205    2200         295.21    114.53   77.69   79.92   75.58    0.78
224    2400        1239.70    219.69   77.36   78.72   76.04    0.77
243    2600          76.18     55.82   77.63   78.69   76.59    0.78
262    2800          86.82     54.30   76.99   78.65   75.39    0.77
281    3000          94.50     67.84   77.55   78.73   76.41    0.78
299    3200          53.94     44.02   77.05   78.49   75.67    0.77
318    3400          62.63     58.89   76.66   78.16   75.21    0.77
336    3600         244.85    201.33   77.31   79.02   75.67    0.77
355    3800          88.47     68.11   76.30   77.42   75.21    0.76
✔ Saved pipeline to output directory
/content/drive/MyDrive/Colab
Notebooks/TopEditor/NERmodels/model_spacy_gpu_trf_es_text_1_7_ALLcat/model-last
```

Salida training con todos los datos disponibles y como base un transformer en espanol

```
============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E    #       LOSS TRANS...  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE
---  ------  -------------  --------  ------  ------  ------  ------
/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
  0       0        9141.63   1037.08    0.00    0.00    0.00    0.00
 12     200      459513.54  121861.23   69.90   73.07   67.00    0.70
 25     400       20726.25  17053.39   76.58   76.76   76.41    0.77
 38     600        6151.98   3596.97   76.79   77.19   76.41    0.77
 50     800        1338.38   1166.75   77.00   78.47   75.58    0.77
 63    1000         724.14    640.02   78.62   80.56   76.77    0.79
 75    1200         888.28    444.03   77.71   78.48   76.96    0.78
 88    1400         531.81    410.78   78.11   80.00   76.31    0.78
100    1600         298.93    323.36   79.38   80.74   78.06    0.79
113    1800         217.09    215.83   78.50   78.94   78.06    0.78
126    2000         187.08    152.38   78.16   79.50   76.87    0.78
138    2200         170.42    145.96   78.71   79.75   77.70    0.79
151    2400         219.99    172.51   77.80   78.75   76.87    0.78
164    2600         122.78    118.62   79.20   80.36   78.06    0.79
176    2800         104.08    106.07   77.80   78.57   77.05    0.78
189    3000         102.84     62.69   78.54   79.70   77.42    0.79
201    3200          74.07     66.11   78.14   79.06   77.24    0.78
✔ Saved pipeline to output directory
/content/drive/MyDrive/Colab
Notebooks/TopEditor/NERmodels/model_spacy_gpu_trf_es_text_all_ALLcat/model-last
```



##Tests del modelo

Con datos que no ha visto en el entrenamiento

In [ ]:
remplacer_mots( \
  '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/test/annot_data_topeditor_15_part_2_modif_OK.conll', \
    ['B-Adefinir30', 'I-Adefinir30','B-expresion_medidas','I-expresion_medidas', 'B-Obras', 'I-Obras', 'B-Religion', 'I-Religion',  'B-estado_casa', 'I-estado_casa', 'B-elemento_arquitectura', 'I-elemento_arquitectura'], \
  ['O', 'O', 'B-medidas', 'I-medidas', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], \
  '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/test/annot_data_topeditor_15_part_2_modif_OK_cleancat.conll')

Les mots ['B-Adefinir30', 'I-Adefinir30', 'B-expresion_medidas', 'I-expresion_medidas', 'B-Obras', 'I-Obras', 'B-Religion', 'I-Religion', 'B-estado_casa', 'I-estado_casa', 'B-elemento_arquitectura', 'I-elemento_arquitectura'] ont été remplacés par ['O', 'O', 'B-medidas', 'I-medidas', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'] dans '/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/test/annot_data_topeditor_15_part_2_modif_OK_cleancat.conll'.


In [ ]:
!python -m spacy convert \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/conll/test/annot_data_topeditor_15_part_2_modif_OK_cleancat.conll" \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/bin_containers/" \
 -t spacy -c iob -b fr_core_news_sm


ℹ Auto-detected token-per-line NER format
⚠ Document delimiters found, automatic document segmentation with `-n`
disabled.
⚠ No sentence boundaries found. Use `-s` to automatically segment
sentences.
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Generated output file (74 documents): /content/drive/MyDrive/Colab
Notebooks/TopEditor/data/bin_containers/annot_data_topeditor_15_part_2_modif_OK_cleancat.spacy


In [25]:
!python -m spacy benchmark accuracy \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/NERmodels/model_spacy_gpu_trf_fr_text_all_ALLcat/model-best" \
"/content/drive/MyDrive/Colab Notebooks/TopEditor/data/bin_containers/annot_data_topeditor_15_part_2_modif_OK_cleancat.spacy"

#--displacy-path 'sample_data/' #'/content/drive/MyDrive/Colab Notebooks/TopEditor/output_html/model_spacy_gpu_trf_fr_text_1_7_ALLcat/'
#Cela ne marche plus.

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/spacy_transformers/layers/hf_shim.py:124: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which u

Salida test modelo 08/12/2024 (textos: 1, 7, 1e partes, 10e partes)



```
================================== Results ==================================

TOK     -    
NER P   85.77
NER R   84.84
NER F   85.30
SPEED   137  


=============================== NER (per type) ===============================

                              P        R        F
Titulo_role_statut        88.98    88.28    88.63
ORG                       84.31    82.69    83.50
LOC                       70.37    73.08    71.70
rol_pers_familia         100.00    60.00    75.00
PER                       89.21    90.51    89.86
tipo_edificio             84.27    92.59    88.24
relations_spatiales       61.76    67.74    64.62
nombre_calle              92.31    80.00    85.71
ExpresionesJuridicas      86.07    75.00    80.15
Oficio                    66.67   100.00    80.00
Fecha_periodos_eventos    96.00    93.51    94.74
modo_pago                 79.31    83.64    81.42
nombre_CTMB               66.67    22.22    33.33
Alias_persona             97.83    91.84    94.74
nombre_collacion          36.36    80.00    50.00
piezas_casas              83.67    82.83    83.25
TipoContratoLT            95.12    97.50    96.30
TipoContratoCT           100.00   100.00   100.00
medidas                   87.50    82.35    84.85

```



Modelo solo textos 1 y 7


```
TOK     -    
NER P   80.63
NER R   78.09
NER F   79.34
SPEED   104  


=============================== NER (per type) ===============================

                              P        R        F
Titulo_role_statut        85.25    81.25    83.20
Oficio                    47.37    90.00    62.07
ORG                       89.80    84.62    87.13
LOC                       66.67    61.54    64.00
rol_pers_familia          50.00    60.00    54.55
PER                       84.78    85.40    85.09
tipo_edificio             86.90    90.12    88.48
relations_spatiales       62.07    58.06    60.00
nombre_calle              81.25    86.67    83.87
ExpresionesJuridicas      69.42    60.00    64.37
Fecha_periodos_eventos    90.14    83.12    86.49
modo_pago                 74.55    74.55    74.55
nombre_CTMB                0.00     0.00     0.00
Alias_persona             78.85    83.67    81.19
nombre_collacion          37.50    60.00    46.15
TipoContratoLT            93.02   100.00    96.39
piezas_casas              90.11    82.83    86.32
estado_casa                0.00     0.00     0.00
TipoContratoCT           100.00   100.00   100.00
Religion                   0.00     0.00     0.00
modo_pago*                 0.00     0.00     0.00
medidas                   82.35    82.35    82.35
elemento_arquitectura    100.00    25.00    40.00
```



Agregando 1/30 del resto de los otros textos (modelo tr ES en base)


```
================================== Results ==================================

TOK     -    
NER P   80.14
NER R   80.84
NER F   80.49
SPEED   104  


=============================== NER (per type) ===============================

                              P        R        F
Titulo_role_statut        82.54    81.25    81.89
ORG                       81.63    76.92    79.21
LOC                       45.45    38.46    41.67
rol_pers_familia         100.00    60.00    75.00
PER                       85.00    86.86    85.92
tipo_edificio             86.36    93.83    89.94
relations_spatiales       48.39    48.39    48.39
nombre_calle              83.87    86.67    85.25
ExpresionesJuridicas      75.00    68.57    71.64
Oficio                    64.29    90.00    75.00
Fecha_periodos_eventos    95.89    90.91    93.33
modo_pago                 76.27    81.82    78.95
nombre_CTMB               20.00    11.11    14.29
Alias_persona             77.19    89.80    83.02
Religion                   0.00     0.00     0.00
nombre_collacion          33.33   100.00    50.00
TipoContratoLT            93.02   100.00    96.39
piezas_casas              88.00    88.89    88.44
TipoContratoPerpetua       0.00     0.00     0.00
estado_casa                0.00     0.00     0.00
TipoContratoCT           100.00   100.00   100.00
modo_pago*                 0.00     0.00     0.00
medidas                   88.24    88.24    88.24
elemento_arquitectura    100.00    25.00    40.00
```



Agregando 1/30 del resto de los otros textos (modelo tr FR en base y menos 2 categorias, estado_casa y Religion, con pocas ocurrencias)



```
================================== Results ==================================

TOK     -    
NER P   82.18
NER R   83.32
NER F   82.75
SPEED   144  


=============================== NER (per type) ===============================

                              P        R        F
Titulo_role_statut        86.29    83.59    84.92
ORG                       89.58    82.69    86.00
LOC                       65.52    73.08    69.09
rol_pers_familia         100.00    60.00    75.00
PER                       87.05    88.32    87.68
tipo_edificio             81.32    91.36    86.05
relations_spatiales       45.45    48.39    46.87
nombre_calle              83.33    83.33    83.33
ExpresionesJuridicas      77.86    72.86    75.28
Oficio                    52.94    90.00    66.67
Fecha_periodos_eventos    96.00    93.51    94.74
modo_pago                 84.21    87.27    85.71
nombre_CTMB               66.67    22.22    33.33
Alias_persona             95.35    83.67    89.13
nombre_collacion          30.77    80.00    44.44
piezas_casas              84.62    88.89    86.70
TipoContratoLT            93.02   100.00    96.39
estado_casa                0.00     0.00     0.00
TipoContratoCT           100.00   100.00   100.00
Religion                   0.00     0.00     0.00
medidas                   80.56    85.29    82.86
elemento_arquitectura    100.00    25.00    40.00

```




##Anotar un texto TEI a partir de un modelo NER entrenado

In [ ]:
#model_spacy_gpu_trf_es = "/NERmodels/model_spacy_gpu_trf_fr_text_all_ALLcat/model-best"

import os
import spacy
from lxml import etree

# Charger le modèle spaCy
model_spacy_gpu_trf_es = "/NERmodels/model_spacy_gpu_trf_fr_text_all_ALLcat/model-best"
gdrive_path = "/content/drive/MyDrive/Colab Notebooks/TopEditor"
nlp = spacy.load(gdrive_path + model_spacy_gpu_trf_es)

# Dossier contenant les fichiers texte
dossier_textes = gdrive_path + '/data/TEI/'

# Fonction pour baliser les entités nommées dans le texte sous forme de sous-éléments XML
def baliser_entites_xml(element, texte, ents):
    dernier_indice = 0

    # Parcourir chaque entité nommée détectée
    for ent in ents:
        # Ajouter le texte avant l'entité directement sans balise <text>
        if dernier_indice < ent.start_char:
            text_before = texte[dernier_indice:ent.start_char]
            element.text = (element.text or '') + text_before  # Ajouter le texte directement sous <p>

        # Ajouter l'entité sous forme de balise XML <ent>
        ent_element = etree.Element("ent", {'type': ent.label_})
        ent_element.text = ent.text
        element.append(ent_element)

        # Mettre à jour l'indice
        dernier_indice = ent.end_char

    # Ajouter le texte restant après la dernière entité, toujours sans balise <text>
    if dernier_indice < len(texte):
        text_after = texte[dernier_indice:]
        element.text = (element.text or '') + text_after

# Fonction pour obtenir le texte complet d'une balise XML
def get_full_text(element):
    return ''.join(element.itertext())

# Lire et traiter les fichiers XML
for fichier in os.listdir(dossier_textes):
    if fichier.endswith('.xml'):
        chemin_fichier = os.path.join(dossier_textes, fichier)

        # Charger et parser le fichier XML avec lxml
        tree = etree.parse(chemin_fichier)
        root = tree.getroot()

        # Namespace TEI
        namespace = {'tei': 'http://www.tei-c.org/ns/1.0'}

        # Trouver le body
        body = root.find('.//tei:body', namespaces=namespace)

        # Extraire les textes dans les balises <p> du body avec leurs attributs xml:id
        paragraphs = body.findall('.//tei:p', namespaces=namespace)

        # Parcourir chaque paragraphe et baliser les entités nommées
        for p in paragraphs:
            # Récupérer le texte du paragraphe
            text = get_full_text(p)

            # Appliquer le modèle spaCy pour détecter les entités nommées
            doc = nlp(text)

            # Effacer le texte existant du paragraphe et baliser les entités
            p.clear()
            baliser_entites_xml(p, text, doc.ents)

        # Sauvegarder le fichier XML modifié avec les balises d'entités
        sortie_fichier = os.path.join(dossier_textes, 'output_08122024_' + fichier)

        # Sauvegarder avec `lxml` et conserver les namespaces
        tree.write(sortie_fichier, encoding="UTF-8", xml_declaration=True, pretty_print=True)

        print(f"Fichier {fichier} traité et sauvegardé sous {sortie_fichier}.")


/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is d

Fichier TOPEDITOR-1-Lequartierdeschanoines-612-634_schemavalideEdith.xml traité et sauvegardé sous /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/output_08122024_TOPEDITOR-1-Lequartierdeschanoines-612-634_schemavalideEdith.xml.
Fichier output3_TOPEDITOR-1-Lequartierdeschanoines-612-634_schemavalideEdith.xml traité et sauvegardé sous /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/output_08122024_output3_TOPEDITOR-1-Lequartierdeschanoines-612-634_schemavalideEdith.xml.
Fichier test_TOPEDITOR-1-Lequartierdeschanoines-612-634_schemavalideEdith.xml traité et sauvegardé sous /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/output_08122024_test_TOPEDITOR-1-Lequartierdeschanoines-612-634_schemavalideEdith.xml.
Fichier test2_TOPEDITOR-1-Lequartierdeschanoines-612-634_schemavalideEdith.xml traité et sauvegardé sous /content/drive/MyDrive/Colab Notebooks/TopEditor/data/TEI/output_08122024_test2_TOPEDITOR-1-Lequartierdeschanoines-612-634_schemavalideEdith.xml.
Fichie

AttributeError: 'NoneType' object has no attribute 'findall'

##TODO

Retirer des categories avec peu d'exemples :
* Low number of examples for label 'TipoContratoCT' (17)
* Low number of examples for label 'elemento_arquitectura' (25)